In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
#!pip install optbinning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 4.5 MB/s 
     |████████████████████████████████| 15.5 MB 18.5 MB/s 
     |████████████████████████████████| 408 kB 58.5 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.3 which is incompatible.
tensorflow-metadata 1.9.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.3 which is incompatible.
google-cloud-bigquery-storage 1.1.2 requires protobuf<4.0.0dev, but you have protobuf 4.21.3 which is incompatible.
google-api-core 1.31.6 requires protobuf<4.

In [5]:
from optbinning import ContinuousOptimalBinning

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
train = pd.read_csv('drive/Shareddrives/DS4W-Team/Data/train.csv')
test = pd.read_csv('drive/Shareddrives/DS4W-Team/Data/test.csv')

In [8]:
train.shape

(75757, 64)

In [9]:
test.shape

(9705, 63)

In [10]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75757 entries, 0 to 75756
Data columns (total 64 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Year_Factor                75757 non-null  int64  
 1   State_Factor               75757 non-null  object 
 2   building_class             75757 non-null  object 
 3   facility_type              75757 non-null  object 
 4   floor_area                 75757 non-null  float64
 5   year_built                 73920 non-null  float64
 6   energy_star_rating         49048 non-null  float64
 7   ELEVATION                  75757 non-null  float64
 8   january_min_temp           75757 non-null  int64  
 9   january_avg_temp           75757 non-null  float64
 10  january_max_temp           75757 non-null  int64  
 11  february_min_temp          75757 non-null  int64  
 12  february_avg_temp          75757 non-null  float64
 13  february_max_temp          75757 non-null  int

In [11]:
train['id'].nunique()

75757

In [12]:
round(train.State_Factor.value_counts()*100/len(train),2)

State_6     67.11
State_11     8.46
State_1      7.42
State_2      6.43
State_4      5.68
State_8      4.89
State_10     0.02
Name: State_Factor, dtype: float64

In [13]:
round(train.building_class.value_counts()*100/len(train),2)

Residential    57.5
Commercial     42.5
Name: building_class, dtype: float64

In [14]:
train.year_built.describe()

count    73920.000000
mean      1952.306764
std         37.053619
min          0.000000
25%       1927.000000
50%       1951.000000
75%       1977.000000
max       2015.000000
Name: year_built, dtype: float64

Year_built does not tell that much itlself, so I create an age variable.


*   Comment: The observations were collected over 7 years as indicated in the year_factor. Therefore, a large "age variable" does not necessarily mean a large age at the time of observation. 

  For example, building A, built in 1950, was observed in 2010 and has an age variable as 72. Building B, built in 1952, was observed in 2015, and has an age variables as 70. However, building B was older than building A when they were observed.

  Therefore, we should be reminded that age variable is the age in 2022, not the age when observed.






In [15]:
train['year_built'] = np.where((train.year_built==0) | (np.isnan(train.year_built)),train.year_built.median(),train.year_built)

In [16]:
train['age'] = 2022 - train['year_built']

In [17]:
train[['year_built', 'age']].describe()

,year_built,age
count,75757.000000,75757.000000
mean,1952.429597,69.570403
std,32.215321,32.215321
min,1600.000000,7.000000
25%,1927.000000,46.000000
50%,1951.000000,71.000000
75%,1976.000000,95.000000
max,2015.000000,422.000000


In [18]:
num = train.select_dtypes(include='number').columns.to_list()

We can't used days_above_110F, see the Elham EDA. The 99.9% quantile is still 0.

In [19]:
train["days_above_110F"].quantile(0.999)

0.0

In [20]:
nv = ['days_above_110F']
feat = list(set(num) - {'id', 'site_eui', 'Year_Factor', 'year_built'} - set(nv))

In [21]:
#http://gnpalencia.org/optbinning/tutorials/tutorial_continuous.html
def IV(data, feat):
    
    full = []
    for i in feat:
        x = data[i].values
        y = data.site_eui
        optb = ContinuousOptimalBinning(name=i, dtype="numerical")
        optb.fit(x, y)
        binning_table = optb.binning_table
        bint = binning_table.build()
        iv = bint.loc[['Totals'],['IV']].reset_index(drop=True)
        full.append([i, iv.to_string(index = False, header=False)])
        
    df = pd.DataFrame(full, columns={'feat','IV'})
    df['IV'] = df['IV'].astype(float)
    df = df.sort_values(by=['IV'])

    return df

In [22]:
full_iv=IV(train, feat)

In [23]:
full_iv

,feat,IV
18,days_above_100F,0.518759
35,days_below_0F,2.327836
15,may_max_temp,2.445293
9,november_max_temp,2.594439
11,days_with_fog,2.608407
23,june_max_temp,2.844517
3,floor_area,2.907236
19,age,2.983570
10,ELEVATION,3.013617
12,july_max_temp,3.470294


For all data less important: days_above_100F, this variable requires more analysis energy_star_rating given the missing

In [24]:
res = train[train.building_class=='Residential']
com = train[train.building_class=='Commercial']

days_above_100F cant be used in residential

In [25]:
res_feat = list(set(feat) - {'days_above_100F'})

In [26]:
res_iv=IV(res, res_feat)
res_iv

,feat,IV
34,days_below_0F,0.598194
14,may_max_temp,1.539028
12,july_max_temp,1.838710
40,october_min_temp,2.105518
10,days_with_fog,2.468372
22,june_max_temp,3.629682
45,november_avg_temp,3.648283
33,august_max_temp,3.705090
3,floor_area,3.739162
31,december_avg_temp,3.783899


In [27]:
com_iv=IV(com, feat)
com_iv

,feat,IV
15,may_max_temp,1.388997
18,days_above_100F,1.914216
43,may_avg_temp,2.957025
53,september_min_temp,3.208061
48,direction_max_wind_speed,3.221654
40,max_wind_speed,3.221654
52,direction_peak_wind_speed,3.320560
8,may_min_temp,3.400961
4,september_max_temp,3.813132
11,days_with_fog,3.851582


In [28]:
st6 = train[train.State_Factor=='State_6']
stn6 = train[train.State_Factor!='State_6']

In [29]:
st6_feat = list(set(feat) - {'days_below_0F', 'days_above_100F'})

st6_iv=IV(st6, st6_feat)
st6_iv

,feat,IV
46,direction_max_wind_speed,0.467749
38,max_wind_speed,0.526848
50,direction_peak_wind_speed,0.632081
11,ELEVATION,0.952225
31,december_avg_temp,1.145210
32,july_min_temp,1.152185
43,december_min_temp,1.221917
41,may_avg_temp,1.221917
52,august_min_temp,1.268514
13,april_max_temp,1.345515


cant be used in state6 'days_below_0F', 'days_above_100F'

In [30]:
stn6_feat = list(set(feat) - {'direction_peak_wind_speed', 'max_wind_speed', 'direction_max_wind_speed'})

stn6_iv=IV(stn6, stn6_feat)
stn6_iv

,feat,IV
15,days_above_100F,1.756614
10,days_with_fog,3.684058
19,age,4.010975
13,april_max_temp,4.474808
6,december_max_temp,4.726350
14,may_max_temp,4.956313
33,july_min_temp,5.609909
12,july_max_temp,5.707489
46,june_avg_temp,6.437804
9,november_max_temp,6.660939


cant be used in other states 'direction_peak_wind_speed', 'max_wind_speed', 'direction_max_wind_speed'